import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Lao động - Tiền lương - Người có công

find elements

In [2]:
# Function to crawl data from a page
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Get author and time information
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Get question title
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Get situation/summary
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Get detail link
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Send HTTP request to get detail page content
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Get answer from detail page
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Add article information to the list
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data

execute function

In [3]:
articles_data = []
for page_num in range(1, 300):
    page_url = f"https://chinhsachonline.chinhphu.vn/lao-dong-tien-luong-nguoi-co-cong/t1/11/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

export to CSV

In [4]:
# Create DataFrame from the data list
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Phạm Thị Thu Hường,08:05 30/07/2025,Công tác ở vùng đặc biệt khó khăn có được nghỉ...,"Tôi sinh ngày 8/5/1971, công tác tại Văn phòng...",Bộ Nội vụ trả lời vấn đề này như sau:Về việc x...
1,Lê Nhã,08:05 29/07/2025,"Cán bộ không chuyên trách xã nghỉ tinh giản, t...","Năm 2017, tôi làm việc tại một công ty, đóng B...",Bộ Nội vụ trả lời vấn đề này như sau:Khoản 5 Đ...
2,Nguyễn Quốc Phú,08:05 28/07/2025,Chế độ với cán bộ không chuyên trách xã đủ tuổ...,"Mẹ tôi sinh năm 1958, làm Chủ tịch Hội Người c...",Bộ Nội vụ trả lời vấn đề này như sau:Sau khi n...
3,Trần Thị Bé Hai,13:05 27/07/2025,"Tiêu chuẩn, hồ sơ, thủ tục xét tặng Bằng khen ...",Tôi là giáo viên trường tiểu học. Theo kết quả...,Bộ Nội vụ trả lời vấn đề này như sau:Khoản 11 ...
4,Lê Thị Ngân,08:05 27/07/2025,Chính sách ưu đãi tiền sử dụng đất đối với con...,Em họ tôi là con liệt sĩ và đang muốn chuyển m...,Đội thuế liên huyện Bình Long – Chơn Thành (cũ...


In [5]:

 # Save DataFrame to CSV file
csv_file = '../source_fine_tuning/data/ld_tl_ncc_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../source_fine_tuning/data/ld_tl_ncc_data.csv


In [6]:
df_bh=pd.read_csv("../source_fine_tuning/data/ld_tl_ncc_data.csv",encoding='utf-8')

In [7]:
df_bh.shape

(3197, 6)